In [71]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms,datasets
from torch import nn,optim
from torchvision.datasets import MNIST

import matplotlib.pyplot as plt
print(torch.cuda.is_available())  # 应该返回True
print(torch.cuda.device_count())  # 返回可用的GPU数量

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
# torch.cuda.amp


True
1
Using device: cuda


In [72]:
# 载入训练集数据，同时把数据转换为tensor，同时下载数据
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

# 载入测试集数据，同时把数据转换为tensor，同时下载数据
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

# 文件分别为训练集测试集他们分别的数据和标签

In [73]:
# 批次大小
batch_size = 64
# 装载数据集,dataloader为数据的装载器，数据来源于dataset=train_dataset, 大小为batch_size=batch_size，方式为随机打乱
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=8)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=8)
# 展示数据
for i,data in enumerate(train_loader):
    inputs, labels = data
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([1000, 1, 28, 28])
torch.Size([1000])


In [74]:
Dop = 0.1 # 0.1的神经元不工作
class Net(nn.Module):
    # 初始化，定义网络结构
    def __init__(self):
        # 初始化nnModule
        super(Net, self).__init__()
        # 定义了一个全连接层，参数为输入值和输出值个数
        # 由于是非线性回归，一层神经网络无法做到，因此需要加入隐藏层
        self.conv1 = nn.Sequential(nn.Conv2d(1, 32, 5,1,2),nn.ReLU(),nn.MaxPool2d(2,2))
        # 输入通道数（黑白1彩色3）、输出通道数（生成32个特征图）、卷积核大小、步长、padding=2，即padding两圈0
        # 加入Relu激活函数，使其非线性
        # 池化为2*2，步长为2
        self.conv2 = nn.Sequential(nn.Conv2d(32, 64, 5,1,2),nn.ReLU(),nn.MaxPool2d(2,2))
        # 64个特征图，大小为7*7（28*28卷积后变成28*28，池化后变成14*14，再次卷积后池化变成7*7）
        self.fc1 = nn.Sequential(nn.Linear(64*7*7,500),nn.Dropout( p = Dop),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(500,100),nn.Dropout( p = Dop),nn.ReLU())
        self.fc3 = nn.Sequential(nn.Linear(100,10),nn.Softmax(dim=1))
        
    # 前向计算，定义网络计算
    def forward(self, x):
        # ([64, 1, 28, 28])
        # 卷积要求四维数据，格式如上
        x = self.conv1(x)
        x = self.conv2(x)
        # ([64, 64, 7, 7])
        # 4维变2维
        x = x.view(x.size()[0],-1)
        x = self.fc1(x)
        x = self.fc2(x)
        outs = self.fc3(x)
        return outs

In [75]:
LR = 0.001
# 定义模型
model = Net()
model.to(device)
# 定义损失函数,交叉熵损失
cross_entropy = nn.CrossEntropyLoss()
# 优化器，设置L2正则化
optimizer = optim.Adam(model.parameters(), lr=LR ,weight_decay=0.0001)

In [76]:
def train():
    # 训练状态
    model.train()
    for i,data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        out = model(inputs)
        # 交叉熵无需使用独热编码
        loss = cross_entropy(out, labels)
        # 梯度清零，计算梯度，修改权值
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def test():
    # 测试状态
    model.eval()
    # 计算测试集的准确率
    correct = 0
    for i, data in enumerate(test_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        out = model(inputs)
        # 计算out中，最大值所在位置
        _, predicted = torch.max(out.data, 1)
        
        correct += (predicted == labels).sum()
    print("Test acc:{0}".format(correct.item() / len(test_loader.dataset)))
    
    
    # 计算训练集的准确率
    correct = 0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        out = model(inputs)
        # 计算out中，最大值所在位置
        _, predicted = torch.max(out.data, 1)
        
        correct += (predicted == labels).sum()
    print("Train acc:{0}".format(correct.item() / len(train_loader.dataset)))
        

In [77]:
for epoch in range(20):
    
    train()
    print('epoch:', epoch,' train over')
    
    test()
    
    if LR >= 0.00001:
        LR = LR * 0.3
    
    
# 一共1w张测试集

epoch: 0  train over
Test acc:0.8485
Train acc:0.8428666666666667
epoch: 1  train over
Test acc:0.9611
Train acc:0.9601
epoch: 2  train over
Test acc:0.9785
Train acc:0.9754833333333334
epoch: 3  train over
Test acc:0.9828
Train acc:0.9819
epoch: 4  train over
Test acc:0.9839
Train acc:0.9831166666666666
epoch: 5  train over
Test acc:0.9837
Train acc:0.9832666666666666
epoch: 6  train over
Test acc:0.9845
Train acc:0.9868166666666667
epoch: 7  train over
Test acc:0.9887
Train acc:0.98895
epoch: 8  train over
Test acc:0.9872
Train acc:0.9907166666666667
epoch: 9  train over
Test acc:0.9894
Train acc:0.9918333333333333
epoch: 10  train over
Test acc:0.9889
Train acc:0.9918833333333333
epoch: 11  train over
Test acc:0.9881
Train acc:0.9911333333333333
epoch: 12  train over
Test acc:0.989
Train acc:0.9919333333333333
epoch: 13  train over
Test acc:0.9918
Train acc:0.9933333333333333
epoch: 14  train over
Test acc:0.9904
Train acc:0.9936


KeyboardInterrupt: 